## Handel quantum 

In [5]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, note, instrument, meter, tempo

# Convert a qubit's reduced state to Bloch sphere angles
def state_to_bloch_angles(alpha, beta):
    theta = 2 * np.arccos(np.abs(alpha))
    phi = np.angle(beta) - np.angle(alpha)
    return theta, phi

# Convert Bloch angles to MIDI notes
def theta_to_note(theta):
    pitch = int((theta / np.pi) * 12)
    return pitch + 60  # Middle C (C4)

def phi_to_note(phi):
    pitch = int(((phi + np.pi) / (2 * np.pi)) * 12)
    return pitch + 72  # One octave above middle C

# Assign instruments to qubits
def qubit_to_instrument(index):
    if index == 0:
        return instrument.Violoncello()  # Use Violoncello for bass line
    instruments = [instrument.Piano(), instrument.Violin(), instrument.Guitar(),
                   instrument.Flute(), instrument.Clarinet()]
    return instruments[index % len(instruments)]

# Apply gate to state using circuit reconstruction
def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

# Main function: quantum circuit to music
def circuit_to_music_explicit(circuit):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            # Record amplitudes at each barrier
            for i in range(num_qubits):
                # Use partial_trace to reduce the state for qubit i
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta))
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    # Repeat the bass line and create variations for the upper voices
    repeat_count = 12  # Number of times to repeat the bass line
    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))

        # Repeat the bass line (first qubit) consistently
        if i == 0:
            for _ in range(repeat_count):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    note_theta = note.Note(theta_to_note(theta), quarterLength=1)
                    part.append(note_theta)  # Bass repeats at a steady rhythm
        else:
            # Upper voices with rhythmic and pitch variation
            for j in range(repeat_count):
                for alpha, beta in qubit_states[i]:
                    theta, phi = state_to_bloch_angles(alpha, beta)
                    # Create rhythmic and melodic variation in upper voices
                    note_theta = note.Note(theta_to_note(theta), quarterLength=0.5 + 0.2 * (j % 3))  # Small rhythmic variation
                    note_phi = note.Note(phi_to_note(phi), quarterLength=0.5 + 0.3 * (j % 2))  # More variation
                    part.append([note_theta, note_phi])  # Higher voices with variation

        music_score.append(part)

    # Set the meter and tempo to match a Baroque passacaglia
    music_score.append(meter.TimeSignature('4/4'))
    music_score.append(tempo.MetronomeMark(number=50 + np.random.randint(0, 10)))  # Slightly varied tempo

    return music_score, qubit_states

# Example: build a more complex quantum circuit with additional gates
qc = qiskit.QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.barrier()
qc.rz(np.pi/4, 2)
qc.ry(np.pi/2, 1)
qc.rx(np.pi/3, 0)
qc.barrier()
qc.ccx(0, 1, 2)
qc.barrier()
qc.cx(1, 2)
qc.rz(np.pi/3, 1)
qc.barrier()

# Add some random operations to increase the variation (you can adjust this part)
for i in range(3):
    qc.u(np.random.rand() * np.pi, np.random.rand() * np.pi, np.random.rand() * np.pi, i)
qc.barrier()

# Generate the musical score
music_score, qubit_states = circuit_to_music_explicit(qc)

# Show the score or save it
music_score.show('text')  # Opens MIDI playback in music21
music_score.write('midi', fp='quantum_music_passacaglia_varied.mid')  # Save to file if desired


{0.0} <music21.stream.Part 0x11df071d0>
    {0.0} <music21.instrument.Violoncello 'Violoncello'>
    {0.0} <music21.note.Note G#>
    {1.0} <music21.note.Note G#>
    {2.0} <music21.note.Note G#>
    {3.0} <music21.note.Note G#>
    {4.0} <music21.note.Note G>
    {5.0} <music21.note.Note G#>
    {6.0} <music21.note.Note G#>
    {7.0} <music21.note.Note G#>
    {8.0} <music21.note.Note G#>
    {9.0} <music21.note.Note G>
    {10.0} <music21.note.Note G#>
    {11.0} <music21.note.Note G#>
    {12.0} <music21.note.Note G#>
    {13.0} <music21.note.Note G#>
    {14.0} <music21.note.Note G>
    {15.0} <music21.note.Note G#>
    {16.0} <music21.note.Note G#>
    {17.0} <music21.note.Note G#>
    {18.0} <music21.note.Note G#>
    {19.0} <music21.note.Note G>
    {20.0} <music21.note.Note G#>
    {21.0} <music21.note.Note G#>
    {22.0} <music21.note.Note G#>
    {23.0} <music21.note.Note G#>
    {24.0} <music21.note.Note G>
    {25.0} <music21.note.Note G#>
    {26.0} <music21.note.Note G#>


'quantum_music_passacaglia_varied.mid'